In [1]:
import numpy as np
from process import get_binary_data
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss,accuracy_score
import scipy as sc
from plotly.offline import init_notebook_mode
import plotly.graph_objects as go
scaler = StandardScaler()
np.random.seed(1)
init_notebook_mode(connected=True)

In [2]:
df_data = pd.read_csv('ecommerce_data.csv')
df_data.head()
display(df_data.dtypes)

is_mobile                 int64
n_products_viewed         int64
visit_duration          float64
is_returning_visitor      int64
time_of_day               int64
user_action               int64
dtype: object

In [3]:
df_data['time_of_day'] = pd.Categorical(df_data['time_of_day'])
X_transformed = pd.get_dummies(df_data)
X_transformed.insert(0,'bias',1)
# split train and test
Xtrain = X_transformed.iloc[:-100,].copy()
Xtest = X_transformed.iloc[-100:,].copy()
Xtrain[['n_products_viewed', 'visit_duration']] = scaler.fit_transform(Xtrain[['n_products_viewed', 'visit_duration']])
Xtest[['n_products_viewed', 'visit_duration']] = scaler.fit_transform(Xtest[['n_products_viewed', 'visit_duration']])
X2_train = Xtrain[Xtrain.user_action <= 1].copy()
display(X2_train.shape)
Y2_train = X2_train.user_action
X2_train.drop(['user_action'],axis =1,inplace =True)
display(X2_train.shape)


X2_test = Xtest[Xtest.user_action <= 1].copy()
display(X2_test.shape)
Y2_test = X2_test.user_action
X2_test.drop(['user_action'],axis =1,inplace =True)
display(X2_test.shape)


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by StandardScaler.

/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by StandardScaler.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by StandardScaler.

/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by StandardScaler.



(321, 10)

(321, 9)

(77, 10)

(77, 9)

In [4]:
test_costs = []
train_costs = []
w_0 = np.random.randn(X2_train.shape[1] -1)
w_0=np.insert(w_0, 0, 0)
l = 0.001
for i in range(10000):
    y_pred = sc.special.expit(X2_train.dot(w_0)).values
    y_train_pred = sc.special.expit(X2_train.dot(w_0))
    y_test_pred = sc.special.expit(X2_test.dot(w_0))
    log_loss_train = log_loss(Y2_train,y_train_pred)
    log_loss_test = log_loss(Y2_test,y_test_pred)
    train_costs.append(log_loss_train)
    test_costs.append(log_loss_test)
    w_new = w_0 -(l*(y_pred  - Y2_train ).dot(X2_train))
    w_0 = w_new

    if i%1000 == 0:
        print(i,log_loss_train,log_loss_test)
print('Final train classfication rate ', accuracy_score(Y2_train,np.round(y_train_pred)))
print('Final test classfication rate ', accuracy_score(Y2_test,np.round(y_test_pred)))

0 1.1820531652094033 1.197550847488557
1000 0.19332568452835147 0.162997036913868
2000 0.19262764036597269 0.1605922616266146
3000 0.19259102128116912 0.1601321092226677
4000 0.192588658387803 0.16002152430252006
5000 0.19258849923265703 0.15999326474513847
6000 0.19258848839855136 0.15998592179884089
7000 0.19258848765903325 0.15998400541115368
8000 0.1925884876085198 0.15998350469173417
9000 0.19258848760506878 0.15998337382322714
Final train classfication rate  0.9688473520249221
Final test classfication rate  0.961038961038961


In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=train_costs,mode ='markers'))
fig.add_trace(go.Scatter(y=test_costs,mode ='markers'))
fig.show()